In [4]:
import os
import json

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [34]:
# user_input = "¿Qué dice el artículo 103 del Código Penal?"  # 1
# user_input = "¿Cuáles son las causales de agravación del homicidio?"  # 2
# user_input = "Si A le roba el celular a B, ¿qué delito comete?"  # 3
# user_input = "¿Cuál es la diferencia entre dolo eventual y culpa con representación?"  # 4
# user_input = "¿Cuáles son las formas de autoría en derecho penal?"  # 5
# user_input = "¿Cuándo se consuma el hurto?"  # 6
# user_input = "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?"  # 7
# user_input = "¿Cuál es el término que tiene la Fiscalía para formular acusación?"  # 8
# user_input = "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"  # 9
# user_input = "¿Se puede precluir una investigación antes de formular imputación?"  # 10
# user_input = "¿Cuáles son las causales de ausencia de responsabilidad en el derecho penal?"  # 11
# user_input = "¿Cuáles son las causales genéricas de atenuación punitiva?"  # 12
# user_input = "¿Qué se entiende por “documento” en el derecho penal?"  # 13
# user_input = "¿Cuáles son las causales de agravación de la estafa?" #14
# user_input = "¿Se pueden realizar preguntas sugestivas en el contrainterrogatorio del juicio oral?" #15
# user_input = "¿Un juez debe ordenar la captura del procesado cuando anuncie sentido de fallo condenatorio?" #16
# user_input = "¿Cuáles son las fuentes de posición de garante según el código penal?" #17
# user_input = "¿La búsqueda selectiva en base de datos requiere control previo?" #18
# user_input = "¿Cuándo procede la captura en flagrancia?" #19
# user_input = "Si alguien falsifica un documento y lo usa como medio para engañar a otra persona y llevarlo a que le pague algo que no le debe, ¿qué delito(s) comete?" #20

# user_input = "Cuál es la sanción por no presentar la declaración de renta a timepo?"
# user_input = "Qué es el derecho a la vida?"
# user_input = "En qué artículo de la constitución está consagrado el derecho a la vida?"
# user_input = "En qué artículo del código penal se contempla la definición de documento?"
# user_input = "¿Qzué dice el artículo 14 del código penal?"
# user_input = "¿Qué dice el artículo 241 del código penal?"
# user_input = "¿Qué se entiende por “documento” en el código penal?"
# user_input = "¿Qué dice el artículo 249 del código penal?"
# user_input = "¿Qué dice el artículo 221 del código penal?"
# user_input = "¿En qué artículo se encuentra el delito de calumnia?"
# user_input = "Si B imputa falsamente a A un delito que no ha cometido ¿En qué conducta punible incurre?"
# user_input = "En qué delito incurre una persona que entre sin autorización en un sistema informático?"
# user_input = "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# user_input = "¿Qué delito comtió una persona que adquirió un bien cuyo origen inmediato es una actividad ilícita?"
user_input = "¿Qué dice el artículo 323 del código penal?"
# user_input = "¿Qué artículo contiene el delito de lavado de activos en el código penal?"
# user_input = "¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?"

print("🧑 User input:")
print(user_input)

🧑 User input:
¿Qué dice el artículo 323 del código penal?


In [6]:
query_vectors = embeddings_client.generate_embeddings(content=user_input)
results_supporting_content = search_client.search(
    search_text=user_input,
    # filter=filters,
    top=3,
    query_type=QueryType.SEMANTIC,
    vector_queries=[
        {
            "vector": query_vectors,
            "k": 3,
            "fields": "content_vector",
            "kind": "vector",
            "exhaustive": True,
        }
    ],
    semantic_configuration_name="default",
    query_caption="extractive|highlight-false",
    scoring_profile="legal",
    scoring_parameters=[
        "tagx6-Constitucional",
        "tagx5-Legal",
        "tagx4-Infralegal",
        "tagx3-Jurisprudencia",
        "tagx2-Doctrina",
    ],
)

results_supporting_content_top = list(results_supporting_content)[:3]

supporting_content = []
printer_supporting_content = []
for document in results_supporting_content_top:
    # captions: QueryCaptionResult = document["@search.captions"]
    # captions_text = "//".join([caption.text for caption in captions])
    doc = {
        # "captions": captions_text,
        # "title": document["title"],
        "keywords": document["keywords"],
        "category": document["category"],
        "content": document["content"],
    }
    supporting_content.append(doc)
    printer_supporting_content.append(json.dumps(doc, indent=4))

print(
    "Supporting content for Query generation - Tokens: ",
    num_tokens_from_string(str(supporting_content)),
)
print("\n".join(printer_supporting_content))

Supporting content for Query generation - Tokens:  1178
{
    "keywords": "Penal",
    "category": "Jurisprudencia",
    "content": "de permiso hasta de 72 horas, no se configura, JEBD fue acusado y condenado por 2 conductas\npor resolver una solicitud elevada directamente de prevaricato por acci\u00f3n agravado, cuando\npor el privado de la libertad, y no por el centro realmente se trataba de una sola. Y si bien, la\nde reclusi\u00f3n / PREVARICATO POR ACCI\u00d3N - decisi\u00f3n que profiri\u00f3 fue manifiestamente\nDelito de ejecuci\u00f3n instant\u00e1nea: cometido a contraria a la ley, la insuficiencia probatoria de\ntrav\u00e9s de varios actos, unidad de acci\u00f3n, cargo conllev\u00f3 a que no se acreditara el dolo en\nexplicaci\u00f3n / CONCURSO HOMOG\u00c9NEO - No se su actuar.\nconfigura ante varios actos que constituyen\nEn este punto, explic\u00f3 el correcto entendimiento\nunidad de acci\u00f3n / ASESORAMIENTO Y\ndel numeral 5\u00b0 del art\u00edculo 38 de la Ley 906 de\

In [35]:
query_system_template = "You are a legal assistant and your goal is to generate a search query for legal documents. You have access to Azure AI Search index with 100's of documents. Generate a search query in Spanish based on what the user asked you. Your query should always contain at least one and no more than two laws, articules and/or documents titles. The query will be used in a hybrid search (semantic and full text) so keep it short but make sure to all the main keywords and actions needed to give the necesary context to the query. If you can't create a quality query out of what the user asked you, you can use the tool call 'get_supporting_info' to identify sources that could help you generate a better query. You response will be in a JSON format following this format: {'query_type':'<query_type>', 'query':<query>} For the 'query_type' you will choose between 'specific' if the user asks for a specific law, article or title, or 'interpretation' if the user didn't asked for something specific but rather need help making an interpretation of the law. Don't include the ```json"


query_user_content_template = f"Generate search query for: {user_input}."

query_messages = [
    {"role": "system", "content": query_system_template},
    {"role": "user", "content": query_user_content_template},
]

In [36]:
get_supporting_info = {
    "type": "function",
    "function": {
        "name": "get_supporting_info",
        "description": "Use this tool to get laws, articles and titles to generate a better query.",
        "parameters": {},
    },
}

In [37]:
import re

query_completion = openai_client.chat.completions.create(
    tools=[get_supporting_info],
    messages=query_messages,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0.0,
    n=1,
)

supporting_content = []
printer_supporting_content = []
query_vectors = []

if query_completion.choices[0].finish_reason == "tool_calls":
    tool_call = query_completion.choices[0].message.tool_calls[0]
    query_vectors = embeddings_client.generate_embeddings(content=user_input)
    results_supporting_content = search_client.search(
        search_text=user_input,
        # filter=filters,
        top=3,
        query_type=QueryType.SEMANTIC,
        vector_queries=[
            {
                "vector": query_vectors,
                "k": 3,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )

    results_supporting_content_top = list(results_supporting_content)[:3]

    for document in results_supporting_content_top:
        doc = {
            # "title": document["title"],
            "keywords": document["keywords"],
            "category": document["category"],
            "content": document["content"],
        }
        supporting_content.append(doc)
        printer_supporting_content.append(json.dumps(doc, indent=4))

    print(
        "Supporting content for Query generation - Tokens: ",
        num_tokens_from_string(str(supporting_content)),
    )
    new_query_user_content_template = f"Generate search query for: {user_input}. Use these supporting documents to identify any useful source to include in the query: {supporting_content}"

    new_query_messages = [
        {"role": "system", "content": query_system_template},
        {"role": "user", "content": query_user_content_template},
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "id": tool_call.id,  # call_rb1Don1fEb6oLpN7rHmBh4RW
                    "function": {
                        "arguments": "{}",
                        "name": "get_supporting_info",
                    },
                    "type": "function",
                }
            ],
        },
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "get_supporting_info",
            "content": str(supporting_content),
        },
    ]
    # print(new_query_messages)
    new_query_completion = openai_client.chat.completions.create(
        messages=new_query_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.0,
        n=1,
    )
    query = new_query_completion.choices[0].message.content
    # query_vectors = embeddings_client.generate_embeddings(content=query)
    print("🔎 Query for:")
    print("no else")
    print(query)
    # print(new_query_completion)
    print("=========")
    print("\n".join(printer_supporting_content))
else:
    print("Else")
    query = query_completion.choices[0].message.content
    # query_vectors = embeddings_client.generate_embeddings(content=query)
    user_input_vectors = embeddings_client.generate_embeddings(content=user_input)

    print("🔎 Query for:")
    print(query)

Supporting content for Query generation - Tokens:  1308
🔎 Query for:
no else
{'query_type':'specific', 'query':'Artículo 323 del Código Penal de Colombia'}
{
    "keywords": "Penal",
    "category": "Legal",
    "content": "LEY 599 DE 2000 C\u00d3DIGO PENAL DE COLOMBIA\n1. A la persona que cometa en el extranjero delito\ncontra la existencia y seguridad del Estado, contra\nel r\u00e9gimen constitucional, contra el orden econ\u00f3mico\nsocial excepto la conducta definida en el art\u00edculo\n323 del presente C\u00f3digo, contra la administraci\u00f3n\np\u00fablica, o falsifique moneda nacional o incurra en el\ndelito de financiaci\u00f3n de terrorismo y administraci\u00f3n\nde recursos relacionados con actividades terroristas,\naun cuando hubiere sido absuelta o condenada en el\nexterior a una pena menor que la prevista en la ley\ncolombiana.\nEn todo caso se tendr\u00e1 como parte cumplida de la pena\nel tiempo que hubiere estado privada de su libertad.\n2. A la persona que est\u00e9 

In [ ]:
def search_specific():
    return search_client.search(
        search_text=query,
        # filter=filters,
        top=50,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )


def search_interpretation():
    return search_client.search(
        search_text=query,
        # filter=filters,
        top=50,
        query_type=QueryType.SEMANTIC,
        vector_queries=[
            {
                # "vector": user_input_vectors,
                "vector": query_vectors,
                "k": 50,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )


# Fields:
# 1. Título primero: 10
# 4. Año: 9
# 3. Autor: 8
# 5. Categoría: 7
# 6. Keywords: 6
# 2. Contenido: 3

# Category:
# "tagx6-Constitucional",
# "tagx5-Legal",
# "tagx4-Infralegal",
# "tagx3-Jurisprudencia",
# "tagx2-Doctrina",


def add_to_context(results, token_limit=3500):
    printer_documents = []

    results_list = list(results)

    top_results = results_list[:5]
    supporting_results = results_list[5:]

    documents = []

    for document in top_results:
        # captions: QueryCaptionResult = document["@search.captions"]
        # print(document)
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "content": document["content"],
            "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }
        documents.append(doc_formatted)
        printer_documents.append(json.dumps(doc_formatted, indent=4))

    current_token_count = num_tokens_from_string(str(documents))

    additional_documents = []
    for document in supporting_results:
        captions: QueryCaptionResult = document["@search.captions"]
        captions_text = " // ".join([caption.text for caption in captions])
        # print(document)
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "captions": captions_text,
            "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }

        # Tema de contexto para responder la respuesta en base a qué
        # pensar en enrutar los queries dependiendo del tema, del keyword.

        new_token_count = current_token_count + num_tokens_from_string(
            str(doc_formatted)
        )

        if new_token_count <= token_limit:
            additional_documents.append(doc_formatted)
            printer_documents.append(json.dumps(doc_formatted, indent=4))
            current_token_count = new_token_count
        else:
            break

    response = {
        "sources": documents + additional_documents,
        "documents": documents,
        "printer_documents": printer_documents,
        "additional_documents": additional_documents,
        "token_count": current_token_count,
    }

    return response


sources = add_to_context(results)
documents = sources["sources"]

print("Legal profile - Query for: ")
print(query)
print("Num of docs:", len(documents), " ---  Tokens: ", sources["token_count"])
print("=============================")
print("\n".join(sources["printer_documents"]))

Legal profile - Query for: 
"Delito juez sentencia manifiestamente contraria a la ley" "Ley 599 de 2000"
Num of docs: 9  ---  Tokens:  3427
{
    "id": "20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30",
    "content": "manifiestamente contraria a la ley, como tambi\u00e9n\nentre la falta de adecuaci\u00f3n t\u00edpica por ausencia\nde adecuarse al delito de peculado por\ndel ingrediente normativo \u201cmanifiestamente\napropiaci\u00f3n al permitir que terceros se\ncontrario a la ley\u201d y la no probanza de \u00abla\napropiaran de bienes de FONCOLPUERTOS,\nconciencia de antijuridicidad\u00bb con la que \u00e9ste\ncuya custodia se le hab\u00eda confiado\u00bb.\nactu\u00f3, tendiendo ambos argumentos a convertir\nen at\u00edpico el comportamiento de su defendido\n(Textos resaltados por la Relator\u00eda)\nfrente al delito de prevaricato por acci\u00f3n.\n10",
    "external_id": "8f0611a4-dd24-4ce0-92e6-f9305221d07e",
    "title": "CSJ - SCP Boletin Jurisprudencial 2018-12-

In [ ]:
response_system_template = f'Eres Ariel, un asistente para la investigación legal \n\n    Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n\nCada fuente tiene un nombre seguido por dos puntos y la información real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. Asegúrate de capturar todo antes de ".pdf", por ejemplo: [SFC - Títulos valores electrónicos. Pagarés. Depósito centralizado de valores. Exigencia Concepto 2020086426-003 del 24 de junio de 2020 id2020086426.pdf]. No dividas los nombres de archivo por ninguna razón, ya que todos deben terminar en .pdf. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'

response_few_shots = []

response_user_content_template = f"{user_input} Sources: {documents}"

response_messages = [
    {"role": "system", "content": response_system_template},
    *query_few_shots,
    {"role": "user", "content": response_user_content_template},
]

# print(response_messages)

In [ ]:
response = (
    openai_client.chat.completions.create(
        messages=response_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.2,
        n=1,
    )
    .choices[0]
    .message.content
)

In [ ]:
print(user_input)
print("===============")
print(response)

¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?
El delito que comete un juez que profiere una sentencia manifiestamente contraria a la ley es el de **prevaricato por acción**. Según el artículo 413 del Código Penal Colombiano (Ley 599 de 2000):

<strong>Artículo 413. Prevaricato por acción.</strong> El servidor público que profiera resolución, dictamen o concepto manifiestamente contrario a la ley, incurrirá en prisión de cuarenta y ocho (48) a ciento cuarenta y cuatro (144) meses, multa de sesenta y seis punto sesenta y seis (66.66) a trescientos (300) salarios mínimos legales mensuales vigentes, e inhabilitación para el ejercicio de derechos y funciones públicas de ochenta (80) a ciento cuarenta y cuatro (144) meses [Ley 599 de 2000.pdf].
